## 1. Gerekli Kütüphanelerin Yüklenmesi

 Bu bölümde projemiz için gerekli olan kütüphaneler yüklenmektedir. Bu kütüphaneler arasında pandas, tensorflow, matplotlib gibi veri işleme, model oluşturma ve görselleştirme için gerekli araçlar bulunmaktadır.

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

## 2. Veri Setinin Yüklenmesi ve Etiketlerin Oluşturulması

Bu bölümde, balık veri setindeki görüntüler klasör yapısı içerisinden taranır ve her bir görüntüye ait etiketler (label) ile yol bilgileri (path) toplanır. Bu veriler daha sonra bir DataFrame'e dönüştürülür.

In [ ]:
label = []
path = []
fish_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'
for dir_name, _, filenames in os.walk(fish_dir):
    for filename in filenames:
        if os.path.splitext(filename)[-1] == '.png':
            if dir_name.split()[-1] != 'GT':
                label.append(os.path.split(dir_name)[-1])
                path.append(os.path.join(dir_name, filename))

data = pd.DataFrame(columns=['path','label'])
data['path'] = path
data['label'] = label

## 3. Kategorilerin ve Görsellerin Görselleştirilmesi

Bu bölümde, veri setindeki farklı balık türleri görselleştirilir. Her bir sınıftan bir örnek görsel ekrana bastırılarak balık türleri arasında görsel bir analiz yapılır.

In [ ]:
directory = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset/Fish_Dataset'

classes = [i for i in os.listdir(directory) if '.' not in i]
classes

In [ ]:
cn = 0

plt.figure(figsize = (20,20))
for unique_label in data['label'].unique():
    plt.subplot(3,3,cn+1)
    plt.imshow(plt.imread(data[data['label'] == unique_label].iloc[0,0]))
    plt.title(unique_label)
    plt.axis('off')
    cn += 1

## 4. Görsellerin Yüklenmesi ve İşlenmesi

Bu bölümde, veri setindeki her bir görüntü belirlenen boyutlara (28x28) küçültülerek işlenir. Görseller normalleştirilir ve daha sonra model için kullanılmak üzere numpy dizisine dönüştürülür.

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def load_images(df, target_size=(28, 28)):
    images = []
    labels = []
    for index, row in df.iterrows():
        img = load_img(row['path'], target_size=target_size)
        img = img_to_array(img)
        img = img / 255.0  # Normalizasyon
        images.append(img)
        labels.append(row['label'])
    return np.array(images), np.array(labels)

X, y = load_images(data)

## 5. Eğitim ve Test Verilerinin Ayrılması

Bu bölümde, veri seti eğitim ve test olarak ayrılır. Etiketler LabelEncoder ile sayısal değerlere çevrilir ve to_categorical ile one-hot encoding formatına dönüştürülür.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [ ]:
# Etiketleri sayısal değerlere çevirme
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)


In [ ]:
# Eğitim ve test setine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)


## 6. Modelin Oluşturulması ve Derlenmesi

Bu bölümde, basit bir yapay sinir ağı (ANN) modeli oluşturulmuştur. İlk katman Flatten ile görüntülerin boyutu düzleştirilir ve ardından gizli katmanlar eklenir. Dropout kullanarak aşırı öğrenme (overfitting) azaltılmaya çalışılır. Son katmanda softmax fonksiyonu kullanılarak sınıflar arasında tahmin yapılır.

In [ ]:
# Modelin oluşturulması
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten


In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28, 3)))  # Görüntü boyutu: 28x28, 3 kanal (RGB)
model.add(Dense(128, activation='relu'))  # Gizli katman
model.add(Dropout(0.5))  # Dropout katmanı
model.add(Dense(64, activation='relu'))  # Gizli katman
model.add(Dense(len(classes), activation='softmax'))  # Çıkış katmanı (Balık sınıfları)



## 7. Modelin Derlenmesi ve Eğitilmesi

Model adam optimizasyon algoritması ve categorical_crossentropy loss fonksiyonu ile derleniyor ve ardından 10 epoch boyunca eğitiliyor.

In [ ]:
# Modelin derlenmesi
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Modeli eğitme
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


## 8. Başarı Metriklerinin Görselleştirilmesi

Accuracy ve loss değerlerinin eğitim ve validasyon süreçleri boyunca değişimi ayrı grafiklerde gösteriliyor.

In [ ]:
# Modelin başarı metriklerini değerlendirme
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Loss grafiği
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Model Loss')

plt.tight_layout()
plt.show()

## 9. Confusion Matrix ve Classification Report

Bu bölümde modelin test seti üzerindeki performansını analiz etmek için confusion matrix ve classification report kullanılıyor.

In [ ]:
# Confusion Matrix ve Classification Report
from sklearn.metrics import confusion_matrix, classification_report

# Tahminler
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

In [ ]:
# Confusion Matrix ve Classification Report
cm = confusion_matrix(y_test_classes, y_pred_classes)
print(classification_report(y_test_classes, y_pred_classes))
sns.heatmap(cm, annot=True, fmt="d")
plt.show()